# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [3]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [4]:
!wget https://dl.fbaipublicfiles.com/fasttext/vector-crawl/cc.ru.300.vec.gz

--2021-05-16 09:02:49--  https://dl.fbaipublicfiles.com/fasttext/vector-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2021-05-16 09:02:50 ERROR 403: Forbidden.



In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/4thCourse/8thSemester/ML/Lab1/cc.uk.300.vec")

In [7]:
uk_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/4thCourse/8thSemester/ML/Lab1/cc.uk.300.vec")

In [9]:
ru_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/4thCourse/8thSemester/ML/Lab1/cc.ru.300.vec")

In [11]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [12]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [13]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [14]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [15]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz

--2021-05-16 09:26:18--  http://tiny.cc/jfgecz
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/jfgecz [following]
--2021-05-16 09:26:18--  https://tiny.cc/jfgecz
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt [following]
--2021-05-16 09:26:18--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Sa

In [16]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez

--2021-05-16 09:26:21--  http://tiny.cc/6zoeez
Resolving tiny.cc (tiny.cc)... 157.245.113.153
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://tiny.cc/6zoeez [following]
--2021-05-16 09:26:21--  https://tiny.cc/6zoeez
Connecting to tiny.cc (tiny.cc)|157.245.113.153|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt [following]
--2021-05-16 09:26:21--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12188 (12K) [text/plain]
Savi

In [17]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [18]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [19]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
# mapping = ...
# -------
mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [20]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531403541564941),
 ('июнь', 0.8402307033538818),
 ('март', 0.8385775089263916),
 ('сентябрь', 0.8331867456436157),
 ('февраль', 0.8311493396759033),
 ('октябрь', 0.8278172016143799),
 ('ноябрь', 0.8244150280952454),
 ('июль', 0.8228995203971863),
 ('август', 0.8112362027168274),
 ('январь', 0.8022859692573547)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [21]:
august

array([[ 3.9489754e-03, -4.3679588e-02, -3.9908528e-02, -3.4676693e-02,
         1.0490743e-03, -3.4156818e-02, -4.2033382e-03,  1.5944395e-02,
         2.7658453e-02,  3.0273486e-02, -6.7144390e-03,  4.4573210e-02,
        -1.1617855e-02,  1.9644728e-02,  2.0160034e-02, -2.7195092e-02,
         5.1068258e-02,  4.1389083e-03, -7.8014024e-02,  3.3080097e-02,
         4.5512330e-02, -5.4459732e-02, -3.0365372e-02,  9.7973924e-04,
         4.2911459e-02,  4.9349725e-02,  3.1893412e-03, -3.1746022e-02,
         3.0854132e-02, -5.6616813e-03, -2.2663757e-02,  1.0958412e-02,
         2.0671241e-02, -8.1699230e-02,  1.3378094e-02, -1.4563064e-02,
        -5.0426118e-02, -1.6701764e-01, -3.2136533e-02,  2.0473193e-02,
         4.1662953e-03, -4.1038971e-02, -3.9050695e-03, -1.2366161e-02,
        -1.4771102e-02,  1.2991340e-03,  3.7981115e-02,  8.5008137e-02,
         2.7710367e-03,  5.4326281e-02,  3.4842443e-02,  1.8740576e-02,
         2.3922408e-02,  2.9232878e-02, -1.4030422e-02,  1.31122

In [22]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        # print(i)
        # print(mapped_vectors[i])
        # print('reshaped: ', mapped_vectors[i].reshape(1, -1))
        top_similar = ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn)
        words = []
        for word in top_similar:
          words.append(word[0])
        # print(words)
        if ru in words:
          num_matches += 1
    # print('num matches: ', num_matches)
    precision_val = num_matches / len(pairs)
    return precision_val

In [23]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [24]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [25]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [26]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [27]:
import numpy as np

In [29]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE GOES HERE
    # compute orthogonal embedding space mapping
    # mapping = ...
    # https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/
    U, S, VT = np.linalg.svd(X_train.T.dot(Y_train))
    mapping = U.dot(VT)
    return mapping

In [30]:
W = learn_transform(X_train, Y_train)

In [31]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.805662989616394),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938188910484314),
 ('февраль', 0.7923861145973206),
 ('декабрь', 0.7715375423431396)]

In [32]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [145]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2021-05-16 10:31:00--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  1.87MB/s    in 0.9s    

2021-05-16 10:31:02 (1.87 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [146]:
!gzip -d ./uk.txt.gz

gzip: ./uk.txt already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [147]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [148]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [ ]:
uk_corpus

In [149]:
# Any necessary preprocessing if needed
# YOUR CODE HERE
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [150]:
vocab = uk_emb.vocab
print(len(vocab))

2000000


In [151]:
words = []
# print(uk_corpus)
for doc in uk_corpus:
    doc = word_tokenize(doc)
    words.append(list(filter(lambda x: x in vocab, doc)))
print(words)
uk_corpus = words

[['Я', 'вже', 'закінчу', 'коледж', ',', 'коли', 'ви', 'вернетеся', 'з', 'Америки', '.'], ['Він', 'наказав', 'мені', 'негайно', 'вийти', 'з', 'кімнати', '.'], ['Як', 'би', 'ти', 'не', 'намагався', ',', 'ти', 'не', 'вивчиш', 'англійську', 'за', 'два-три', 'місяці', '.'], ['Поки', 'я', 'не', 'подзвонив', ',', 'він', 'не', 'прийшов', '.'], ['У', 'всесвіті', 'багато', 'галактик', '.'], ['Вона', 'приймає', 'душ', 'щоранку', '.'], ['Неслухняний', 'хлопчик', 'заблукав', 'й', 'оглядався', 'по', 'сторонах', '.'], ['Вона', 'повільно', 'зникала', 'в', 'туманному', 'лісі', '.'], ['Наш', 'літак', 'летів', 'понад', 'хмарами', '.'], ['У', 'Майка', 'є', 'декілька', 'друзів', 'у', 'Флориді', '.'], ['Місто', 'бомбардували', 'ворожі', 'літаки', '.'], ['Я', 'зустрінуся', 'з', 'тобою', 'в', 'неділю', 'о', 'третій', '.'], ['Для', 'фінансування', 'війни', 'було', 'видано', 'облігації', '.'], ['Звідки', 'беруть', 'початок', 'Олімпійські', 'ігри', '?'], ['Я', 'сказав', 'собі', ':', '«', 'Це', 'гарна', 'ідея', '

In [174]:
def translate(sentence, tokenized=0):
    # Handling unseen words in the word2vec/doc2vec model
    # https://github.com/RaRe-Technologies/gensim/issues/310
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
    translated = []
    if tokenized == 1:
      tokenized_sentence = sentence
    else:
      tokenized_sentence = word_tokenize(sentence)
    for word in tokenized_sentence:
      # transform ukrainian embedding vector
      uk_transformed = np.matmul(uk_emb[word], W)
      # find nearest russian word
      most_similar = ru_emb.most_similar([uk_transformed], topn=1)
      # print(most_similar)
      translated.append(most_similar[0][0])
    return " ".join(translated)

In [175]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [176]:
for sent in uk_corpus[::10]:
  print(sent)
  print(translate(sent, 1))

['Я', 'вже', 'закінчу', 'коледж', ',', 'коли', 'ви', 'вернетеся', 'з', 'Америки', '.']
Я уже закончу колледж , когда мы прибежишь со Америки .
['Місто', 'бомбардували', 'ворожі', 'літаки', '.']
Город бомбили враждебные самолеты .
['Можливо', ',', 'я', 'антисоціальний', ',', 'але', 'це', 'не', 'означає', ',', 'що', 'я', 'не', 'спілкуюся', 'з', 'людьми', '.']
Возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми .
['Цього', 'ранку', 'випала', 'роса', '.']
Впрочем утра выпала роса .
['Біда', 'не', 'приходить', 'одна', '.']
Беда не приходит одна .
['Подивися', 'на', 'той', 'дим', '.']
Посмотри по тот дым .
['Я', 'замовив', 'два', 'гамбургера', '.']
Я заказал два гамбургера .
['Я', 'не', 'хотів', 'нікого', 'образити', '.']
Я не хотел никого обидеть .
['Гора', 'вкрита', 'снігом', '.']
Гора покрыта снегом .
['На', 'фотографії', 'в', 'дівчини', 'корона', 'не', 'з', 'золота', ',', 'а', 'з', 'квітів', '.']
по фотографии во девушки корона не со золота , а со цве

Great! 
See second notebook for the Neural Machine Translation assignment.